In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
from matplotlib import pyplot as plt

SAMPLE_TRAINING_SET.xlsx
TEST.xlsx
TRAINING.xlsx



In [2]:
sts = pd.read_excel('../input/SAMPLE_TRAINING_SET.xlsx')
test = pd.read_excel('../input/TEST.xlsx')
train = pd.read_excel('../input/TRAINING.xlsx')

In [3]:
sts.head()

,ID,Q201601_MILES,Q201601_SPONSORS,Q201602_MILES,Q201602_SPONSORS,Q201603_MILES,Q201603_SPONSORS,Q201604_MILES,Q201604_SPONSORS,FLAG,...,MILES201603CREDIT_CARD,MILES201604CREDIT_CARD,MILES201601OTHER,MILES201602OTHER,MILES201603OTHER,MILES201604OTHER,PROGRAM_TIER,PROVINCE,EMAIL_OPTIN,DEC_2016_ACCOUNT_BALANCE
0,1,58,1,36,2,38,2,72,2,0,...,0,16,0,1,1,0,B,ON,Y,266
1,2,43,4,153,4,186,3,246,4,1,...,174,219,7,5,0,4,B,?,N,28
2,4,291,8,455,4,711,7,338,8,0,...,465,123,123,112,205,165,G,AB,Y,6085
3,5,117,4,110,4,113,3,119,3,0,...,0,0,83,84,89,78,B,BC,Y,416
4,7,1222,4,2154,4,1405,4,1858,7,0,...,1367,1811,15,24,37,45,O,AB,N,13111


In [4]:
X = train
# y = train['FLAG']

combined = pd.concat([X, test], axis=0)
combined_dict = {"X": X, "test": test}
combined.head()

,DEC_2016_ACCOUNT_BALANCE,EMAIL_OPTIN,FLAG,ID,MILES201601CREDIT_CARD,MILES201601GROCERY,MILES201601OTHER,MILES201602CREDIT_CARD,MILES201602GROCERY,MILES201602OTHER,...,PROGRAM_TIER,PROVINCE,Q201601_MILES,Q201601_SPONSORS,Q201602_MILES,Q201602_SPONSORS,Q201603_MILES,Q201603_SPONSORS,Q201604_MILES,Q201604_SPONSORS
0,266,Y,0.0,1,0,58,0,0,35,1,...,B,ON,58,1,36,2,38,2,72,2
1,28,N,1.0,2,29,7,7,126,22,5,...,B,?,43,4,153,4,186,3,246,4
2,6085,Y,0.0,4,135,33,123,342,1,112,...,G,AB,291,8,455,4,711,7,338,8
3,416,Y,0.0,5,0,34,83,0,26,84,...,B,BC,117,4,110,4,113,3,119,3
4,13111,N,0.0,7,1201,6,15,2130,0,24,...,O,AB,1222,4,2154,4,1405,4,1858,7


In [5]:
for dataset in combined_dict:
    print(combined_dict[dataset].shape)

(33887, 26)
(11500, 25)


In [6]:
train['PROGRAM_TIER'].value_counts()

B    23054
G     9510
O     1295
?       28
Name: PROGRAM_TIER, dtype: int64

In [7]:
train['PROVINCE'].value_counts()

ON    11158
QC     7708
AB     4416
BC     3237
NS     2049
MB     1381
NB     1272
NL     1002
SK      803
?       592
PE      238
NT       17
YT       14
Name: PROVINCE, dtype: int64

In [8]:
temp = train.loc[train['PROGRAM_TIER'] != '?']
temp['PROGRAM_TIER'].value_counts()


B    23054
G     9510
O     1295
Name: PROGRAM_TIER, dtype: int64

In [9]:
def maptiers(v):
    if v == "B":
        return 1
    if v == "G":
        return 2
    else:
        return 3
temp['PROGRAM_TIER'].apply(lambda x: maptiers(x)).value_counts()
# ['PROGRAM_TIER'].value_counts()

1    23054
2     9510
3     1295
Name: PROGRAM_TIER, dtype: int64

In [10]:
# define prov_dict to use to map values
prov_dict = {
    'ON'  :  1,
    'QC'   :  2,
    'AB'   :  3,
    'BC'   :  4,
    'NS'   :  5,
    'MB'   :  6,
    'NB'   :  7,
    'NL'   :  8,
    'SK'   :   9,
    '?'    :   10,
    'PE'   :   11,
    'NT'   :    12,
    'YT'   :    13
}

#map function:
def map_prov(v):
    retval = -1
    try:
        retval = prov_dict[v]
    except:
        pass
    return retval

In [11]:
temp['PROVINCE'].apply(lambda x: map_prov(x)).value_counts()


1     11158
2      7708
3      4416
4      3237
5      2049
6      1381
7      1272
8      1002
9       803
10      564
11      238
12       17
13       14
Name: PROVINCE, dtype: int64

In [12]:
for key in combined_dict.keys():
    dataset = combined_dict[key]
    dataset = dataset.loc[dataset['PROGRAM_TIER'] != '?']
    dataset['PROVINCE'] = dataset['PROVINCE'].apply(lambda x: map_prov(x))
    dataset['PROGRAM_TIER'] = dataset['PROGRAM_TIER'].apply(lambda x: maptiers(x))
    dataset['EMAIL_OPTIN'] = dataset['EMAIL_OPTIN'].apply(lambda x: x == 'Y').astype(int)
    combined_dict[key] = dataset.drop('ID', axis=1)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [13]:
# At this point I would try to visualize and engineer new features, but lack experience and time
# So will try to train some models to see what happens

In [14]:
combined_dict['X'].columns

Index(['Q201601_MILES', 'Q201601_SPONSORS', 'Q201602_MILES',
       'Q201602_SPONSORS', 'Q201603_MILES', 'Q201603_SPONSORS',
       'Q201604_MILES', 'Q201604_SPONSORS', 'FLAG', 'MILES201601GROCERY',
       'MILES201602GROCERY', 'MILES201603GROCERY', 'MILES201604GROCERY',
       'MILES201601CREDIT_CARD', 'MILES201602CREDIT_CARD',
       'MILES201603CREDIT_CARD', 'MILES201604CREDIT_CARD', 'MILES201601OTHER',
       'MILES201602OTHER', 'MILES201603OTHER', 'MILES201604OTHER',
       'PROGRAM_TIER', 'PROVINCE', 'EMAIL_OPTIN', 'DEC_2016_ACCOUNT_BALANCE'],
      dtype='object')

In [15]:
# import sklearn stuff
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [16]:
# split train test, would normally include a validation set, but time constraints and limited experience don't allow for this
X_train, X_test, y_train, y_test = train_test_split(combined_dict['X'].drop(['FLAG'], axis=1), combined_dict['X']['FLAG'], test_size=0.33, random_state=42)


In [17]:
# clf = RandomForestClassifier()
# clf.fit(X_train, y_train)
# rf_cm = confusion_matrix(clf.predict(X_test), y_test)
# rf_acc = accuracy_score(clf.predict(X_test), y_test)

# clf = SVC()
# clf.fit(X_train, y_train)
# svc_cm = confusion_matrix(clf.predict(X_test), y_test)
# svc_acc = accuracy_score(clf.predict(X_test), y_test)

# clf = LinearSVC()
# clf.fit(X_train, y_train)
# lsvc_cm = confusion_matrix(clf.predict(X_test), y_test)
# lsvc_acc = accuracy_score(clf.predict(X_test), y_test)

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# lr_cm = confusion_matrix(clf.predict(X_test), y_test)
# lr_acc = accuracy_score(clf.predict(X_test), y_test)

# clf = KNeighborsClassifier(n_neighbors = 3)
# clf.fit(X_train, y_train)
# knn_cm = confusion_matrix(clf.predict(X_test), y_test)
# knn_acc = accuracy_score(clf.predict(X_test), y_test)

# clf = GaussianNB()
# clf.fit(X_train, y_train)
# nb_cm = confusion_matrix(clf.predict(X_test), y_test)
# nb_acc = accuracy_score(clf.predict(X_test), y_test)

# clf = Perceptron()
# clf.fit(X_train, y_train)
# p_cm = confusion_matrix(clf.predict(X_test), y_test)
# p_acc = accuracy_score(clf.predict(X_test), y_test)

# clf = SGDClassifier()
# clf.fit(X_train, y_train)
# sgd_cm = confusion_matrix(clf.predict(X_test), y_test)
# sgd_acc = accuracy_score(clf.predict(X_test), y_test)

# clf = DecisionTreeClassifier()
# clf.fit(X_train, y_train)
# dt_cm = confusion_matrix(clf.predict(X_test), y_test)
# dt_acc = accuracy_score(clf.predict(X_test), y_test)



In [18]:
# Looks like RandomForestClassifier is the best option, like usual
# The perceptron might perform better if made into a deep neural net
models = pd.DataFrame({
    'Model': ['RandomForestClassifier', 'SVC', 'LinearSVC', 'LogisticRegression', 'KNeighborsClassifier', 'GaussianNB', 'Perceptron', 'SGDClassifier', 'DecisionTreeClassifier'],
    'Score': [rf_acc, svc_acc, lsvc_acc, 
              lr_acc, knn_acc, nb_acc, 
              p_acc, sgd_acc, dt_acc]})
models.sort_values(by='Score', ascending=False)

NameError: name 'rf_acc' is not defined

In [19]:
# At this point, I would try optimizing the tree but don't have much experience for a competive environment
# So I will make a prediction now, and then submit


#  Retrain model on entire training dataset
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
rf_cm = confusion_matrix(clf.predict(X_test), y_test)
rf_acc = accuracy_score(clf.predict(X_test), y_test)

test_pred = clf.predict(combined_dict['test'])


In [20]:
id_vals = test.loc[test['PROGRAM_TIER'] != '?']['ID']
    
final_out = pd.DataFrame({"CUSTOMER_ID":id_vals, "DECLINER_FLAG":test_pred})  

In [21]:
# save to excel
writer = pd.ExcelWriter('./output.xlsx')
final_out.to_excel(writer)
writer.save()

In [22]:
os.listdir()

NameError: name 'os' is not defined